# **Informe comparativo sobre algoritmos en Java y Python**

In [1]:
# Importacion de modulos requeridos
import pandas as pd

In [42]:
# Leer el CSV
df_python_results = pd.read_csv('../NPC_SUB_DIGRAFH_PYTHON_CODE/results.csv', sep=';', encoding='utf-8')
df_java_results = pd.read_csv('../NPC_SUB_DIGRAFH_JAVA_CODE/results.csv', sep=';', encoding='utf-8')

## **Verificación de que los datos fueron cargados correctamente**

In [43]:
# Ver las primeras filas
print(df_python_results.head(2))

print('-----------------------------------')
print('Datos Java')
print(df_python_results.head(2))

  vertices                                           aristas  n  k  \
0  [1...8]  [(1, 3), (1, 2), (2, 6), (4, 6), (6, 7), (7, 8)]  6  2   
1  [1...8]  [(1, 3), (1, 2), (2, 6), (4, 6), (6, 7), (7, 8)]  6  2   

              metodo                 tiempo  
0  BipartiteSubsetBF  5.080000264570117e-05  
1  BipartiteSubsetDC  4.840000183321535e-05  
-----------------------------------
Datos Java
  vertices                                           aristas  n  k  \
0  [1...8]  [(1, 3), (1, 2), (2, 6), (4, 6), (6, 7), (7, 8)]  6  2   
1  [1...8]  [(1, 3), (1, 2), (2, 6), (4, 6), (6, 7), (7, 8)]  6  2   

              metodo                 tiempo  
0  BipartiteSubsetBF  5.080000264570117e-05  
1  BipartiteSubsetDC  4.840000183321535e-05  


In [44]:
df_python_results.info()
df_java_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   vertices  30 non-null     object
 1   aristas   30 non-null     object
 2   n         30 non-null     int64 
 3   k         30 non-null     int64 
 4   metodo    30 non-null     object
 5   tiempo    30 non-null     object
dtypes: int64(2), object(4)
memory usage: 1.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   vertices   10 non-null     object 
 1   aristas    10 non-null     object 
 2   n          10 non-null     int64  
 3   k          10 non-null     int64  
 4   tiempo BF  10 non-null     float64
 5   tiempo DC  10 non-null     float64
 6   tiempo BT  10 non-null     float64
dtypes: float64(3), int64(2), object(2)
memory usage: 692.0+ bytes


## **Grafica comparativa de tiempo Python vs Java**